### N Gram In N Gram Analysis

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [44]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # 28  # native word end index

# adding native word to shared word
twogram_start = 0  # 0  # native word start index
twogram_end = 200  # 28  # native word end index

# sentence check
twogram_sentence_check = False  # True, False
threegram_sentence_check = False
fourgram_sentence_check = False
fivegram_sentence_check = False

# n gram sample
threegram_sample = 6
fourgram_sample = 4
fivegram_sample = 2

# n gram select
twogram_select_start = 0
twogram_select_end = 10000

threegram_select_start = 0
threegram_select_end = 10000

fourgram_select_start = 0
fourgram_select_end = 10000

fivegram_select_start = 0
fivegram_select_end = 10000

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [45]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/5-N Gram In N Gram Analysis/{lang_folder.capitalize()} {lang_pair.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [46]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [3]:
def word_in_wordgroup_simple(df, source_column, target_column, word_sample_num):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [48]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [49]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [50]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [51]:
df_pair = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

,dict_entry_main
0,robot
1,sistematik
2,prens
3,albüm
4,ekstrem
...,...
594,ritüel
595,enteresan
596,yoğurt
597,burjuva


In [52]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair = pd.merge(df_pair,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair.drop_duplicates(inplace=True)
    df_pair.reset_index(drop=True, inplace=True)
    df_pair = df_pair.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair.rename(columns={"word":"dict_entry_main"}, inplace=True)
else:
    pass

if etymology_word:
    df_pair
else:
    df_pair = df_pair.head(0)
    
df_pair

,dict_entry_main


In [53]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [54]:
word_select = df_word["word"].values.tolist()
words = df_pair["dict_entry_main"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [55]:
word_list = list((word_select_set.union(words_set)).difference(disable_word_set))
len(word_list)

200

In [56]:
if twogram_sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram = df_twogram_sent.loc[:,["twogram","frequency"]]
    #df_twogram_select = df_twogram.iloc[twogram_start:twogram_end,]
else:
    df_twogram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")  
    df_twogram = df_twogram.loc[:,["twogram","frequency"]]
    #df_twogram_select = df_twogram.iloc[twogram_start:twogram_end,]

df_twogram

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
4457029,fikret cibran,3
4457030,romalı fikret,3
4457031,fikret ciooney,3
4457032,fikret cisco,3


In [57]:
d_list2  = df_twogram.iloc[:,0].values.tolist()

resultlist2 = []
manager = multiprocessing.Manager()
resultlist2 = manager.list()

def word_in_wordgroup(d_list2):
    mergelist = []
    try:
        word = d_list2.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist2.append(d_list2)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list2) # string_word liste

result_list2 = list(resultlist2)
df_result2 = pd.DataFrame(result_list2, columns=[0])  # add columns parameter for empty result
df_result2 = df_result2.rename(columns = {0: "twogram"})
df_merge2 = pd.merge(df_result2, df_twogram, how="left", on="twogram")
df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
df_merge_result2.drop_duplicates(inplace=True)
df_merge_result2.reset_index(drop=True, inplace=True)
df_twogram_select = df_merge_result2.iloc[twogram_start:twogram_end,]
df_twogram_select

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
195,ama o,27599
196,daha da,27502
197,bu iş,27459
198,bu değil,27366


In [58]:
if threegram_sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram = df_threegram_sent.loc[:,["threegram","frequency"]]
else:
    df_threegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")  
    df_threegram = df_threegram.loc[:,["threegram","frequency"]]

df_threegram

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
3009751,haydi büyük fikret,5
3009752,fikret caesardan beri,5
3009753,fikret cage kazandı,5
3009754,haydi bakayım fikret,5


In [59]:
d_list3  = df_threegram.iloc[:,0].values.tolist()

resultlist3 = []
manager = multiprocessing.Manager()
resultlist3 = manager.list()

def word_in_wordgroup(d_list3):
    mergelist = []
    try:
        word = d_list3.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist3.append(d_list3)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list3) # string_word liste

result_list3 = list(resultlist3)
df_result3 = pd.DataFrame(result_list3, columns=[0])  # add columns parameter for empty result
df_result3 = df_result3.rename(columns = {0: "threegram"})
df_merge3 = pd.merge(df_result3, df_threegram, how="left", on="threegram")
df_merge_result3 = df_merge3.sort_values(by="frequency", ascending=False)
df_merge_result3.drop_duplicates(inplace=True)
df_merge_result3.reset_index(drop=True, inplace=True)
df_threegram_select = df_merge_result3
df_threegram_select

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,başka bir şey,75193
4,hayır hayır hayır,69443
...,...,...
109758,anne de seni,10
109759,olan biri ama,10
109760,pekala bu o,10
109761,anne de bir,10


In [60]:
if fourgram_sentence_check:
    df_fourgram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Four_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fourgram_sent.rename(columns={"four_gram":"fourgram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fourgram = df_fourgram_sent.loc[:,["fourgram","frequency"]]
else:
    df_fourgram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fourgram_Merge.csv")  
    df_fourgram = df_fourgram.loc[:,["fourgram","frequency"]]

df_fourgram

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
3052209,fikret miloya merhaba de,5
3052210,fikret millsin oğlu mu,5
3052211,fikret millet iyi iş,5
3052212,fikret millet bize bakıyor,5


In [61]:
d_list4  = df_fourgram.iloc[:,0].values.tolist()

resultlist4 = []
manager = multiprocessing.Manager()
resultlist4 = manager.list()

def word_in_wordgroup(d_list4):
    mergelist = []
    try:
        word = d_list4.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist4.append(d_list4)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list4) # string_word liste

result_list4 = list(resultlist4)
df_result4 = pd.DataFrame(result_list4, columns=[0])  # add columns parameter for empty result
df_result4 = df_result4.rename(columns = {0: "fourgram"})
df_merge4 = pd.merge(df_result4, df_fourgram, how="left", on="fourgram")
df_merge_result4 = df_merge4.sort_values(by="frequency", ascending=False)
df_merge_result4.drop_duplicates(inplace=True)
df_merge_result4.reset_index(drop=True, inplace=True)
df_fourgram_select = df_merge_result4
df_fourgram_select

,fourgram,frequency
0,bir şey var mı,41773
1,bir sorun mu var,21423
2,bu da ne demek,16884
3,en son ne zaman,15141
4,başka bir şey var,12619
...,...,...
79278,biliyor musun ben her,7
79279,biliyor musun ben orada,7
79280,biliyor musun iyi ki,7
79281,biliyor musun hiçbir şey,7


In [62]:
if fivegram_sentence_check:
    df_fivegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Five_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fivegram_sent.rename(columns={"five_gram":"fivegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fivegram = df_fivegram_sent.loc[:,["fivegram","frequency"]]
else:
    df_fivegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fivegram_Merge.csv")  
    df_fivegram = df_fivegram.loc[:,["fivegram","frequency"]]

df_fivegram

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
1096831,peder fikret hep şöyle söylerdi,4
1096832,peder fikret intihar etmeye çalışıyor,4
1096833,fikret dolson 12 gün yaşadı,4
1096834,ama fikret diye biri yoktu,4


In [63]:
d_list5  = df_fivegram.iloc[:,0].values.tolist()

resultlist5 = []
manager = multiprocessing.Manager()
resultlist5 = manager.list()

def word_in_wordgroup(d_list5):
    mergelist = []
    try:
        word = d_list5.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist5.append(d_list5)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list5) # string_word liste

result_list5 = list(resultlist5)
df_result5 = pd.DataFrame(result_list5, columns=[0])  # add columns parameter for empty result
df_result5 = df_result5.rename(columns = {0: "fivegram"})
df_merge5 = pd.merge(df_result5, df_fivegram, how="left", on="fivegram")
df_merge_result5 = df_merge5.sort_values(by="frequency", ascending=False)
df_merge_result5.drop_duplicates(inplace=True)
df_merge_result5.reset_index(drop=True, inplace=True)
df_fivegram_select = df_merge_result5
df_fivegram_select

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,bu da ne demek şimdi,4791
4,hayır hayır hayır hayır hayır,4380
...,...,...
16040,tamam bana birkaç saat ver,10
16041,evet ama ben çok daha,10
16042,evet ama ben de bunun,10
16043,bu güzel bir şey çünkü,10


In [64]:
df_ngram_concat = pd.concat([df_twogram_select, df_threegram_select, df_fourgram_select, df_fivegram_select], axis=1)
df_ngram_concat

,twogram,frequency,threegram,frequency,fourgram,frequency,fivegram,frequency
0,bir şey,859944.0,bir şey yok,113165,bir şey var mı,41773.0,başka bir şey var mı,14104.0
1,değil mi,585879.0,bir şey var,110455,bir sorun mu var,21423.0,bu da ne demek oluyor,10205.0
2,ben de,377765.0,bu da ne,89463,bu da ne demek,16884.0,o kadar da kötü değil,7012.0
3,teşekkür ederim,370619.0,başka bir şey,75193,en son ne zaman,15141.0,bu da ne demek şimdi,4791.0
4,ne oldu,322758.0,hayır hayır hayır,69443,başka bir şey var,12619.0,hayır hayır hayır hayır hayır,4380.0
...,...,...,...,...,...,...,...,...
109758,NaN,NaN,anne de seni,10,NaN,NaN,NaN,NaN
109759,NaN,NaN,olan biri ama,10,NaN,NaN,NaN,NaN
109760,NaN,NaN,pekala bu o,10,NaN,NaN,NaN,NaN
109761,NaN,NaN,anne de bir,10,NaN,NaN,NaN,NaN


In [65]:
df_two_in_three = word_in_wordgroup_simple(df_ngram_concat, "twogram","threegram",threegram_sample)
df_two_in_four = word_in_wordgroup_simple(df_ngram_concat, "twogram","fourgram",fourgram_sample)
df_two_in_five = word_in_wordgroup_simple(df_ngram_concat, "twogram","fivegram",fivegram_sample)

In [66]:
df_twogram_order_join_threegram = df_two_in_three.groupby(["twogram"])["threegram"].apply(", ".join).reset_index()
df_twogram_order_join_fourgram = df_two_in_four.groupby(["twogram"])["fourgram"].apply(", ".join).reset_index()
df_twogram_order_join_fivegram = df_two_in_five.groupby(["twogram"])["fivegram"].apply(", ".join).reset_index()  

In [67]:
dfs = [df_twogram_order_join_threegram, df_twogram_order_join_fourgram, df_twogram_order_join_fivegram]

In [68]:
df_ngram_sample_join_merge = reduce(lambda  left,right: pd.merge(left,right, on=['twogram'], how='outer'), dfs)
df_ngram_sample_join_merge.drop_duplicates(inplace=True)
df_ngram_sample_join_merge 

,twogram,threegram,fourgram,fivegram
0,al bakalım,"al bakalım dostum, tamam al bakalım, de al bak...","gel de al bakalım, al bakalım bu senin, mı ist...","al bakalım bu senin için, hayır o değil al bak..."
1,ama ben,"ama ben de, evet ama ben, ama ben bir, ama ben...","ama ben yine de, ama ben bir şey, ama ben hiçb...","ama ben senin gibi değilim, ama ben hiçbir şey..."
2,ama bu,"evet ama bu, ama bu çok, ama bu bir, ama bu ge...","ama bu doğru değil, ama bu sadece bir, ama bu ...","ama bu iyi bir şey, ama bu demek değil ki"
3,ama o,"ama o kadar, ama o zaman, evet ama o, ama o da...","ama o kadar da, ama o zaman da, ama o sadece b...","ama o kadar da değil, ama o kadar da kötü"
4,bana bak,"bana bak bana, bak bana bak, hey bana bak, şim...","bana bak bana bak, bir de bana bak, bana bak t...","bak bir de bana bak, bak bana bak bana bak"
...,...,...,...,...
195,şey değil,"bir şey değil, şey değil mi, aynı şey değil, h...","önemli bir şey değil, bir şey değil mi, başka ...","bu iyi bir şey değil, hayır öyle bir şey değil"
196,şey mi,"bir şey mi, şey mi var, şey mi oldu, hiçbir şe...","bir şey mi var, bir şey mi oldu, kötü bir şey ...","bu iyi bir şey mi, bu kötü bir şey mi"
197,şey var,"bir şey var, şey var mı, çok şey var, her şey ...","bir şey var mı, başka bir şey var, tek bir şey...","başka bir şey var mı, yeni bir şey var mı"
198,şey yok,"bir şey yok, hiçbir şey yok, şey yok mu, şey y...","diye bir şey yok, başka bir şey yok, bir şey y...","biz diye bir şey yok, başka bir şey yok mu"


In [69]:
df_ngram_sample_join_merge = pd.merge(df_ngram_sample_join_merge, df_twogram_select, how="left", on="twogram")
df_ngram_sample_join_merge.drop_duplicates(inplace=True)
df_ngram_sample_join_merge.rename(columns={"frequency":"two_freq"}, inplace=True)
df_ngram_sample_join_merge.sort_values(by="two_freq", ascending=False, inplace=True)
df_ngram_sample_join_merge.reset_index(drop=True, inplace=True)
df_ngram_sample_join_merge

,twogram,threegram,fourgram,fivegram,two_freq
0,bir şey,"bir şey yok, bir şey var, başka bir şey, bir ş...","bir şey var mı, başka bir şey var, bir şey dah...","başka bir şey var mı, daha önce böyle bir şey",859944
1,değil mi,"öyle değil mi, biliyorsun değil mi, güzel deği...","de öyle değil mi, bir şey değil mi, çok güzel ...","ne kadar güzel değil mi, güzel bir gün değil mi",585879
2,ben de,"ben de seni, ben de öyle, ve ben de, evet ben ...","o zaman ben de, ben de aynı şeyi, evet ben de ...","sen de biliyorsun ben de, ben de sana aynı şeyi",377765
3,teşekkür ederim,"çok teşekkür ederim, için teşekkür ederim, teş...","için çok teşekkür ederim, şey için teşekkür ed...","her şey için teşekkür ederim, teşekkür ederim ...",370619
4,ne oldu,"sonra ne oldu, ne oldu sana, sana ne oldu, ne ...","sana ne oldu böyle, ne oldu sana böyle, ne old...","sonra ne oldu biliyor musun, ve ne oldu biliyo...",322758
...,...,...,...,...,...
195,ama o,"ama o kadar, ama o zaman, evet ama o, ama o da...","ama o kadar da, ama o zaman da, ama o sadece b...","ama o kadar da değil, ama o kadar da kötü",27599
196,daha da,"daha da kötü, bir daha da, daha da iyi, daha d...","ve bir daha da, her şeyi daha da, her gün daha...","bu her şeyi daha da, belki de daha da önce",27502
197,bu iş,"bu iş için, bu iş böyle, bu iş çok, bu iş bura...","bu iş için çok, bu iş bu kadar, bu iş için en,...","bu iş bana göre değil, bu iş için en iyi",27459
198,bu değil,"bu değil mi, şey bu değil, sorun bu değil, ola...","da bu değil mi, önemli olan bu değil, şey bu d...","olan da bu değil mi, önemli olan bu değil mi",27366


In [70]:
df_ngram_sample_join_merge.to_excel(f"{twogram_end}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_With_{word_end}_Word_Join_Result.xlsx", index=False)

#### Adding Frequency For Ratio

In [71]:
df_two_in_three_freq = pd.merge(df_two_in_three, df_threegram, how="left", on="threegram")
df_two_in_three_freq.drop_duplicates(inplace=True)
df_two_in_three_freq.rename(columns={"frequency":"three_freq"}, inplace=True)
df_two_in_three_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_three_freq

,threegram,three_freq
0,bir şey yok,113165
1,bir şey var,110455
2,başka bir şey,75193
3,bir şey değil,68498
4,bir şey mi,64933
...,...,...
1195,çok önemli bir,11035
1196,bu önemli bir,2437
1197,daha önemli bir,1948
1198,için önemli bir,1492


In [72]:
df_two_in_four_freq = pd.merge(df_two_in_four, df_fourgram, how="left", on="fourgram")
df_two_in_four_freq.drop_duplicates(inplace=True)
df_two_in_four_freq.rename(columns={"frequency":"four_freq"}, inplace=True)
df_two_in_four_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_four_freq

,fourgram,four_freq
0,bir şey var mı,41773
1,başka bir şey var,12619
2,bir şey daha var,12185
3,bir şey mi var,11834
4,de öyle değil mi,5379
...,...,...
795,de bu değil mi,711
796,önemli bir şey değil,6650
797,çok önemli bir şey,2348
798,önemli bir şey var,1417


In [73]:
df_two_in_five_freq = pd.merge(df_two_in_five, df_fivegram, how="left", on="fivegram")
df_two_in_five_freq.drop_duplicates(inplace=True)
df_two_in_five_freq.rename(columns={"frequency":"five_freq"}, inplace=True)
df_two_in_five_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_five_freq

,fivegram,five_freq
0,başka bir şey var mı,14104
1,daha önce böyle bir şey,2160
2,ne kadar güzel değil mi,1647
3,güzel bir gün değil mi,1423
4,sen de biliyorsun ben de,296
...,...,...
392,bu iş için en iyi,106
393,olan da bu değil mi,228
394,önemli olan bu değil mi,202
395,çok önemli bir şey var,457


In [74]:
df_ngram_sample_concat = pd.concat([df_twogram_select,df_two_in_three_freq, df_two_in_four_freq, df_two_in_five_freq], axis=1)
df_ngram_sample_concat

,twogram,frequency,threegram,three_freq,fourgram,four_freq,fivegram,five_freq
0,bir şey,859944.0,bir şey yok,113165,bir şey var mı,41773.0,başka bir şey var mı,14104.0
1,değil mi,585879.0,bir şey var,110455,başka bir şey var,12619.0,daha önce böyle bir şey,2160.0
2,ben de,377765.0,başka bir şey,75193,bir şey daha var,12185.0,ne kadar güzel değil mi,1647.0
3,teşekkür ederim,370619.0,bir şey değil,68498,bir şey mi var,11834.0,güzel bir gün değil mi,1423.0
4,ne oldu,322758.0,bir şey mi,64933,de öyle değil mi,5379.0,sen de biliyorsun ben de,296.0
...,...,...,...,...,...,...,...,...
1195,NaN,NaN,çok önemli bir,11035,NaN,NaN,NaN,NaN
1196,NaN,NaN,bu önemli bir,2437,NaN,NaN,NaN,NaN
1197,NaN,NaN,daha önemli bir,1948,NaN,NaN,NaN,NaN
1198,NaN,NaN,için önemli bir,1492,NaN,NaN,NaN,NaN


In [75]:
df_ngram_sample_concat.to_excel(f"{twogram_end}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_With_{word_end}_Word_Frequency_Result.xlsx", index=False)

#### Calculate Result And Select Ratio

In [76]:
df_ngram_threegram_unique = df_ngram_sample_concat[["threegram","three_freq"]].drop_duplicates()
df_ngram_fourgram_unique = df_ngram_sample_concat[["fourgram","four_freq"]].drop_duplicates()
df_ngram_fivegram_unique = df_ngram_sample_concat[["fivegram","five_freq"]].drop_duplicates()

In [77]:
twogram_result_freq = df_twogram_select["frequency"].sum()
threegram_result_freq = df_ngram_threegram_unique["three_freq"].sum()
fourgram_result_freq = df_ngram_fourgram_unique["four_freq"].sum()
fivegram_result_freq = df_ngram_fivegram_unique["five_freq"].sum()

In [78]:
twogram_select_freq = df_twogram.iloc[twogram_select_start:twogram_select_end,]["frequency"].sum()
threegram_select_freq = df_threegram.iloc[threegram_select_start:threegram_select_end,]["frequency"].sum()
fourgram_select_freq = df_fourgram.iloc[fourgram_select_start:fourgram_select_end,]["frequency"].sum()
fivegram_select_freq = df_fivegram.iloc[fivegram_select_start:fivegram_select_end,]["frequency"].sum()

In [79]:
(twogram_result_freq/twogram_select_freq)*100

26.08395756606318

In [80]:
(threegram_result_freq/threegram_select_freq)*100

24.59831983959042

In [81]:
(fourgram_result_freq/fourgram_select_freq)*100

15.018156650707319

In [82]:
(fivegram_result_freq/fivegram_select_freq)*100

7.338480135351901

#### Copy Move And Delete

In [42]:
output_file = glob.glob(f"{twogram_end}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_*_Result.xlsx")
output_file

['1000_Twogram_In_6_Threegram_4_Fourgram_2_Fivegram_Sample_With_200_Word_Join_Result.xlsx',
 '1000_Twogram_In_6_Threegram_4_Fourgram_2_Fivegram_Sample_With_200_Word_Frequency_Result.xlsx']

In [43]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [44]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

In [17]:
df_twogram_three = pd.read_excel("twogram_selected.xlsx")
df_twogram_three

,twogram,search_string
0,biz de,biz de öyle
1,biz de,biz de seni
2,yok artık,yok artık daha
3,yok artık,yok artık ya
4,yok artık,gerek yok artık
...,...,...
199,oh evet,oh evet efendim
200,oh evet,oh evet ben
201,ne zaman,ne zaman oldu
202,hayır efendim,hayır efendim ben


In [25]:
df_twogram_three["twogram"].nunique()

61

In [18]:
df_threegram_result = pd.read_excel("Turkish_183_Threegram_With_28_And_172_Word_Pair_Selected_6_Youtube_0.6s_Timeshift_Result.xlsx")
df_threegram_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,hayır doğru değil,8080,8082,hayır doğru değil,4RMoZo69RE8,https://www.youtube.com/watch?v=4RMoZo69RE8&t=...
1,hayır doğru değil,993,996,hayır doğru değil,Isab2NO9jlk,https://www.youtube.com/watch?v=Isab2NO9jlk&t=...
2,hayır doğru değil,5833,5835,yok hayır doğru değil,Ik61qAqcnXo,https://www.youtube.com/watch?v=Ik61qAqcnXo&t=...
3,hayır doğru değil,33,36,hayır doğru değil,PeQifrSZmBo,https://www.youtube.com/watch?v=PeQifrSZmBo&t=33s
4,peki ya sen,387,390,peki ya sen,Qs1I7HxXYlg,https://www.youtube.com/watch?v=Qs1I7HxXYlg&t=...
...,...,...,...,...,...,...
929,belki daha sonra,5983,5985,belki daha sonra oynarsınız olur mu,g81mNCrBnO4,https://www.youtube.com/watch?v=g81mNCrBnO4&t=...
930,belki daha sonra,2261,2264,canım belki daha sonra bahçeye bir şey yapacaktır,WLGk7o3R5Es,https://www.youtube.com/watch?v=WLGk7o3R5Es&t=...
931,belki daha sonra,421,423,canım belki daha sonra bahçeye bir şey yapacaktır,ccnwNHUwrP0,https://www.youtube.com/watch?v=ccnwNHUwrP0&t=...
932,belki daha sonra,2993,2995,belki daha sonra seni bir yerlere götürürüm,X48jgyzoQdE,https://www.youtube.com/watch?v=X48jgyzoQdE&t=...


In [19]:
df_threegram_result["search_string"].nunique()

183

In [21]:
df_two_in_three = pd.merge(df_twogram_three, df_threegram_result, how="inner", on="search_string")
df_two_in_three.drop_duplicates(inplace=True)
df_two_in_three

,twogram,search_string,start_time,end_time,sentence,video_id,video_url
0,biz de,biz de öyle,5741,5743,e biz de öyle badegülle karşılaşmıştık,MJtGR4MDI4M,https://www.youtube.com/watch?v=MJtGR4MDI4M&t=...
1,biz de,biz de öyle,3085,3087,biz de öyle diyoruz da,OEMr2FXddsY,https://www.youtube.com/watch?v=OEMr2FXddsY&t=...
2,biz de,biz de öyle,8953,8955,yani biz de öyle şeyler bir tek filmlerde oluy...,VxK7lhI_2pc,https://www.youtube.com/watch?v=VxK7lhI_2pc&t=...
3,biz de,biz de öyle,1334,1335,yigit i leride biz de öyle baba anne,Aii8sJ5Uw_8,https://www.youtube.com/watch?v=Aii8sJ5Uw_8&t=...
4,biz de,biz de öyle,2833,2835,bizden öncekiler nasıl yaptıysa nasıl kaldılar...,259wx6vTLz4,https://www.youtube.com/watch?v=259wx6vTLz4&t=...
...,...,...,...,...,...,...,...
949,hayır efendim,hayır efendim bu,6459,6463,hayır efendim bu,f694PFHVMFM,https://www.youtube.com/watch?v=f694PFHVMFM&t=...
950,hayır efendim,hayır efendim bu,2022,2024,hayır efendim bu kez tam tersi seyirciye saygı...,YedJhuZQqpI,https://www.youtube.com/watch?v=YedJhuZQqpI&t=...
951,hayır efendim,hayır efendim bu,4860,4862,bu taraf pelinin hayır efendim bu taraf bizim,n4dSJgjRUAM,https://www.youtube.com/watch?v=n4dSJgjRUAM&t=...
952,hayır efendim,hayır efendim bu,4754,4757,hayır efendim bu oyuncak bu mağazanın en çok s...,bt7g4VoVBoE,https://www.youtube.com/watch?v=bt7g4VoVBoE&t=...


In [22]:
df_two_in_three["search_string"].nunique()

183

In [23]:
df_two_in_three["twogram"].nunique()

59

In [26]:
df_two_in_three.to_excel("Turkish_183_Threegram_With_28_And_172_Word_Pair_Selected_6_Youtube_0.6s_Timeshift_Result2.xlsx")